In [1]:
'''

    wine 분류
    
    1.0 4898 - white wine
    0.0 1599 - red wine
    
'''

'\n\n    wine 분류\n    \n    1.0 4898 - white wine\n    0.0 1599 - red wine\n    \n'

In [2]:
import numpy as np
import pandas as pd

### 1. 데이터 준비

In [3]:
wine = pd.read_csv(r"C:\Users\YB\Desktop\tf24_study\01_지도학습\kNN_wine분류_사이킷런API활용_수강생제공용\wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
wine.shape

(6497, 4)

In [5]:
wine['class'].value_counts()

1.0    4898
0.0    1599
Name: class, dtype: int64

In [12]:
### label을 문자열로 인위적으로 변경
wine['label'] = ['W' if x== 1 else 'R' for x in wine['class']]
wine


,alcohol,sugar,pH,class,label
0,9.4,1.9,3.51,0.0,R
1,9.8,2.6,3.20,0.0,R
2,9.8,2.3,3.26,0.0,R
3,9.8,1.9,3.16,0.0,R
4,9.4,1.9,3.51,0.0,R
...,...,...,...,...,...
6492,11.2,1.6,3.27,1.0,W
6493,9.6,8.0,3.15,1.0,W
6494,9.4,1.2,2.99,1.0,W
6495,12.8,1.1,3.34,1.0,W


### 2. 입력데이터와 label 생성

In [13]:
wine_data = wine.iloc[:,:3]
wine_target = wine.iloc[:,3]
wine_data.shape, wine_target.shape

((6497, 3), (6497,))

### 3. 훈련 데이터 및 테스트 데이터 분리

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test = train_test_split(wine_data,wine_target,test_size=0.2,random_state=1)

In [16]:
X_train.shape,y_train.shape

((5197, 3), (5197,))

In [17]:
X_test.shape,y_test.shape

((1300, 3), (1300,))

### 결정트리는 표준화 작업이 필요없다. 매우 큰 장점이다. 

### 결정트리는 대신 과적합에 매우 민감하다 매우 큰 단점이다. (가지치기 -> 하이퍼파라미터로 과적합 방지)

### 4. 모델 생성

In [18]:
!pip install lightgbm

In [19]:
#from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [20]:
#조기종료
'''
    eval_set(검증데이터)에 지정된 데이터가 
    eval_metric에 설정한 특정 평가지표인 loss가 
    특정 횟수만큼 실행되었는데도 불구하고 감소하지 않으면 중단되는 것.
    => 검증데이터가 10번 반복하는데도 손실이 감소안하면 멈춰라
    

'''

clf= LGBMClassifier(learning_rate=0.1,
                  n_estimators=200,
                  random_state=1,
                  max_depth=3,
                  n_jobs=-1,
                  eval_metric="logloss" #손실함수 
)

clf.fit(X_train,y_train,
        eval_set=[(X_test,y_test)],
        early_stopping_rounds=10,
        verbose=1) #순차처리이기 때문에 n_jobs가 지원안됨

[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.503014
[2]	valid_0's binary_logloss: 0.474906
[3]	valid_0's binary_logloss: 0.452701
[4]	valid_0's binary_logloss: 0.433455
[5]	valid_0's binary_logloss: 0.418814
[6]	valid_0's binary_logloss: 0.406115
[7]	valid_0's binary_logloss: 0.393724
[8]	valid_0's binary_logloss: 0.383283
[9]	valid_0's binary_logloss: 0.375483
[10]	valid_0's binary_logloss: 0.367485
[11]	valid_0's binary_logloss: 0.360845
[12]	valid_0's binary_logloss: 0.355
[13]	valid_0's binary_logloss: 0.349806
[14]	valid_0's binary_logloss: 0.344995
[15]	valid_0's binary_logloss: 0.341086
[16]	valid_0's binary_logloss: 0.337622
[17]	valid_0's binary_logloss: 0.334266
[18]	valid_0's binary_logloss: 0.331392
[19]	valid_0's binary_logloss: 0.328383
[20]	valid_0's binary_logloss: 0.325849
[21]	valid_0's binary_logloss: 0.323837
[22]	valid_0's binary_logloss: 0.322006
[23]	valid_0's binary_logloss: 0.320366
[24]	valid_0's binary_logloss: 0.318523


C:\Users\YB\anaconda3\envs\tf24\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\YB\anaconda3\envs\tf24\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(eval_metric='logloss', max_depth=3, n_estimators=200,
               random_state=1)

### 5. 평가

In [21]:
clf.score(X_train,y_train)

0.8799307292668848

In [22]:
clf.score(X_test,y_test)

0.8784615384615385